In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import gc
import warnings
from random import choice
import numpy as np
import pandas as pd
import nltk
import random
import nltk
import gensim.downloader as api
import torch
import torch.nn as nn
import datasets
from sklearn.utils import shuffle
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')
sns.set_palette('Set2')

In [ ]:
!pip install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 30.1 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 59.2 MB/s 
     |████████████████████████████████| 212 kB 77.9 MB/s 
     |████████████████████████████████| 140 kB 72.5 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 75.3 MB/s 
     |████████████████████████████████| 127 kB 74.5 MB/s 
     |████████████████████████████████| 271 kB 76.2 MB/s 
     |████████████████████████████████| 144 kB 47.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 112 kB 77.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing in

Загружаем данные

In [ ]:
dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/experiment/train.csv', 'test': '/content/drive/MyDrive/experiment/test.csv'})

Using custom data configuration default-8abd0c9fa5406f10


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8abd0c9fa5406f10/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

--2022-06-09 17:31:21--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  8.16MB/s    in 4.3s    

2022-06-09 17:31:27 (5.91 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



Загружаем эмюеддинги из navec

In [ ]:
from navec import Navec

path = 'navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
SEED = 0xDEAD
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

In [ ]:
MAX_LENGTH=128

tokenizer = nltk.WordPunctTokenizer()

dataset = dataset.map(
    lambda item: {
        "tokenized": tokenizer.tokenize(item["only_text"])[:MAX_LENGTH]
    }
)

  0%|          | 0/14151 [00:00<?, ?ex/s]

  0%|          | 0/6970 [00:00<?, ?ex/s]

In [ ]:
word2vec = navec.as_gensim
word2idx = {word: idx for idx, word in enumerate(word2vec.index2word)}

In [ ]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["<unk>"]

In [ ]:
dataset = dataset.map(
    lambda item: {
        "features": [encode(word) for word in item["tokenized"]]
    }
)

  0%|          | 0/14151 [00:00<?, ?ex/s]

  0%|          | 0/6970 [00:00<?, ?ex/s]

In [ ]:
dataset["train"][0]

{'features': [144256, 250000, 132253, 250000],
 'only_text': 'отлично, нравится!',
 'rating': 2,
 'tokenized': ['отлично', ',', 'нравится', '!']}

In [ ]:
dataset = dataset.remove_columns(["only_text", "tokenized"])

In [ ]:
dataset.set_format(type='torch')

In [ ]:
dataset['train'][0]

{'features': tensor([144256, 250000, 132253, 250000]), 'rating': tensor(2)}

In [ ]:
def collate_fn(batch):
    max_len = max(len(row["features"]) for row in batch)
    input_embeds = torch.empty((len(batch), max_len), dtype=torch.long)
    labels = torch.empty(len(batch), dtype=torch.long)
    for idx, row in enumerate(batch):
        to_pad = max_len - len(row["features"])
        input_embeds[idx] = torch.cat((row["features"], torch.zeros(to_pad)))
        labels[idx] = row["rating"]
    return {"features": input_embeds, "labels": labels}

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    k: DataLoader(
        ds, shuffle=(k=="train"), batch_size=32, collate_fn=collate_fn
    ) for k, ds in dataset.items()
}

CNN(MaxPooling) without embeddings

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=3):
        super().__init__()
        self.embeddings = nn.Embedding(250002, embedding_dim=embed_size)
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = self.embeddings(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(300, 50, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10

In [ ]:
from tqdm.notebook import tqdm, trange

def training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2):
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        acc,f1 = predict_test(model, loaders)
        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs} /// Accuracy: {acc}, F1: {f1}")

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


def predict_test(model, loaders):
    y_pred = []
    y_test = []
    model.eval()
    with torch.no_grad():
        for batch in loaders["test"]:
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            y_pred.append(prediction.argmax(-1).cpu().numpy())
            y_test.append(labels.cpu().numpy())

            
    y_pred = [x for xs in y_pred for x in xs]
    y_test = [x for xs in y_test for x in xs]
    

    return(accuracy_score(y_test, y_pred),f1_score(y_test, y_pred, average='macro'))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.79261314868927, accuracy: 0.6129124760627747 /// Accuracy: 0.6129124820659971, F1: 0.5314498321000443


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.7584596276283264, accuracy: 0.6384505033493042 /// Accuracy: 0.6384505021520803, F1: 0.6376380243714904


...

Valid Loss: 1.7703700065612793, accuracy: 0.6172166466712952 /// Accuracy: 0.6172166427546628, F1: 0.6012233613086213


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.8111486434936523, accuracy: 0.6073170304298401 /// Accuracy: 0.6073170731707317, F1: 0.6048911680984227


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.9050041437149048, accuracy: 0.6239597797393799 /// Accuracy: 0.6239598278335724, F1: 0.6186694818362928


CNN(MaxPooling) with embeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(300, 50, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10

In [ ]:
# используем navec embeddings
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embeddings.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.7471892833709717, accuracy: 0.6474891901016235 /// Accuracy: 0.6474892395982783, F1: 0.6124009096373868


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.7468690872192383, accuracy: 0.6582496166229248 /// Accuracy: 0.6582496413199426, F1: 0.6326045033779817


...

Valid Loss: 1.5887235403060913, accuracy: 0.6296986937522888 /// Accuracy: 0.6296987087517933, F1: 0.6234492086258482


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.5669273138046265, accuracy: 0.6232424378395081 /// Accuracy: 0.6232424677187949, F1: 0.6175671052341198


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.8351839780807495, accuracy: 0.6331419944763184 /// Accuracy: 0.633142037302726, F1: 0.62085272788316


CNN(AvgPooling) without embeddings

In [ ]:
class CNNModel_AvgPooling(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=3):
        super().__init__()
        self.embeddings = nn.Embedding(250002, embedding_dim=embed_size)
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = self.embeddings(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel_AvgPooling(300, 50, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.7694612741470337, accuracy: 0.6351506114006042 /// Accuracy: 0.6351506456241033, F1: 0.6058807241685057


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.7858039736747742, accuracy: 0.6483500599861145 /// Accuracy: 0.6483500717360114, F1: 0.6250051885639559


...

Valid Loss: 2.066328525543213, accuracy: 0.636298418045044 /// Accuracy: 0.6362984218077475, F1: 0.6177688696204546


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.9535869359970093, accuracy: 0.633859395980835 /// Accuracy: 0.6338593974175036, F1: 0.6313443310894155


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 2.0926170349121094, accuracy: 0.6304160356521606 /// Accuracy: 0.630416068866571, F1: 0.6264871503003975


CNN(AvgPooling) with embeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel_AvgPooling(300, 50, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10

In [ ]:
# используем navec embeddings
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embeddings.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 3.3885750770568848, accuracy: 0.38909611105918884 /// Accuracy: 0.3890961262553802, F1: 0.2920649749777518


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.8016784191131592, accuracy: 0.6552367210388184 /// Accuracy: 0.6552367288378766, F1: 0.6555503840732863


...

Valid Loss: 6.167093276977539, accuracy: 0.4612625241279602 /// Accuracy: 0.4612625538020086, F1: 0.4109947861461974


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.672706127166748, accuracy: 0.6317073106765747 /// Accuracy: 0.6317073170731707, F1: 0.6251263638770712


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 2.0737459659576416, accuracy: 0.6352940797805786 /// Accuracy: 0.6352941176470588, F1: 0.6209457573217735


RNN without embeddings

In [ ]:
class RNN(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_h = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_h = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_x = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_x = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden=None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device)
        seq_length = x.size(1)
        for cur_idx in range(seq_length):
            hidden = torch.tanh(
                x[:, cur_idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h
            )
        return hidden

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=3):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embed_size)
        self.rnn = RNN(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embeddings(x)
        hidden = self.rnn(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 5
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.0985276699066162, accuracy: 0.3671448826789856 /// Accuracy: 0.36714490674318506, F1: 0.1821589055298715


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1361485719680786, accuracy: 0.34203729033470154 /// Accuracy: 0.34203730272596844, F1: 0.20074434188189091


...

Valid Loss: 1.095653772354126, accuracy: 0.3680057227611542 /// Accuracy: 0.3680057388809182, F1: 0.18249812915733077


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1276963949203491, accuracy: 0.36814919114112854 /// Accuracy: 0.36814921090387376, F1: 0.18168156178010367


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1088221073150635, accuracy: 0.3430415987968445 /// Accuracy: 0.3430416068866571, F1: 0.20213940101162178


RNN with embeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 5
max_grad_norm = 1.0

In [ ]:
# используем navec embeddings
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embeddings.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1760427951812744, accuracy: 0.2951219379901886 /// Accuracy: 0.2951219512195122, F1: 0.2188013818891207


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1116039752960205, accuracy: 0.34203729033470154 /// Accuracy: 0.34203730272596844, F1: 0.27205486385877925


...

Valid Loss: 1.1800627708435059, accuracy: 0.35193684697151184 /// Accuracy: 0.35193687230989956, F1: 0.21812007449754112


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1386929750442505, accuracy: 0.3654232323169708 /// Accuracy: 0.3654232424677188, F1: 0.21746796537281043


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1483770608901978, accuracy: 0.3418938219547272 /// Accuracy: 0.34189383070301294, F1: 0.20028205226720686


RNN(GRU) without embeddings

In [ ]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_rh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_rh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_rx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_zh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_zh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_zx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_nh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_nh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_nx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device)
        
        for cur_idx in range(x.size(1)):
            r = torch.sigmoid(
                x[:, cur_idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh
            )
            z = torch.sigmoid(
                x[:, cur_idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh
            )
            n = torch.tanh(
                x[:, cur_idx] @ self.w_nx + self.b_nx + r * (hidden @ self.w_nh + self.b_nh)
            )
            hidden = (1 - z) * n + z * hidden

        return hidden

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=3):
        super().__init__()
        self.embed = nn.Embedding(len(word2idx), embed_size)
        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embed(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.0304958820343018, accuracy: 0.45954087376594543 /// Accuracy: 0.4595408895265423, F1: 0.4170085065333791


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.0180249214172363, accuracy: 0.4703012704849243 /// Accuracy: 0.4703012912482066, F1: 0.41709467950895407


...

Valid Loss: 1.0052794218063354, accuracy: 0.5383070111274719 /// Accuracy: 0.5383070301291248, F1: 0.5424414631983848


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.0415388345718384, accuracy: 0.5398852229118347 /// Accuracy: 0.5398852223816356, F1: 0.5365186208871028


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.077231764793396, accuracy: 0.5451936721801758 /// Accuracy: 0.5451936872309899, F1: 0.5172397906215614


RNN(GRU) with embeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 10
max_grad_norm = 1.0

In [ ]:
# используем navec embeddings
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1114320755004883, accuracy: 0.34031563997268677 /// Accuracy: 0.34031563845050217, F1: 0.2766341197452736


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 1.1183713674545288, accuracy: 0.37288376688957214 /// Accuracy: 0.372883787661406, F1: 0.2848289419522068


...

Valid Loss: 0.9589449763298035, accuracy: 0.5500717163085938 /// Accuracy: 0.5500717360114777, F1: 0.5507973450609378


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.9629761576652527, accuracy: 0.5616929531097412 /// Accuracy: 0.5616929698708751, F1: 0.5458528320508869


  0%|          | 0/443 [00:00<?, ?it/s]

Valid Loss: 0.9850540161132812, accuracy: 0.5579626560211182 /// Accuracy: 0.5579626972740316, F1: 0.5478834663944233
